In [1]:
import warnings, os
warnings.filterwarnings("ignore")

import pandas as pd
import numpy as np
from glob import glob


import matplotlib.pyplot as plt
import matplotlib.image as mpimg

from aggmap import AggMap, loadmap
import seaborn as sns


import xgboost as xgb
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.model_selection import KFold, StratifiedKFold
from sklearn.metrics import confusion_matrix, precision_recall_curve, roc_auc_score
from sklearn.metrics import auc as calculate_auc


## LGR, RF, KNN, ROF, XGBoost

In [2]:
from sklearn.ensemble import RandomForestClassifier
from rotation_forest import RotationForestClassifier

In [3]:
def score(dfr):
    y_true = dfr.y_true
    y_score = dfr.y_score
    y_pred = dfr.y_score.round()

    tn, fp, fn, tp = confusion_matrix(y_true, y_pred).ravel()

    acc = (tp + tn) / sum([tn, fp, fn, tp])
    auc = roc_auc_score(y_true, y_score)
    ber =  (fp / (fp + tn) + fn / (tp + fn)) * 0.5

    print('acc: %.3f, roc-auc: %.3f, BER: %.3f' % (acc,auc,ber))

    return acc, auc


def get_best_params(X, y):
    clf = xgb.XGBClassifier(eval_metric='logloss', silent=True, verbose=0)
    parameters = {'max_depth': [4, 6, 9],
                  'n_estimators': [20, 50, 100]}

    grid = GridSearchCV(clf, parameters, scoring='roc_auc', n_jobs=9, iid=False, cv=5, verbose=0)
    gres = grid.fit(X, y)
    return grid.best_params_

In [4]:
mp = loadmap('/raid/shenwanxiang/agg_mp_object/TCGA-S-G.mp')


In [5]:
lst = glob('/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/*.h5')

n_fold = 5

res = []
for p in lst:
    
    print('#'*50 + ' %s ' % p + '#'*50 )
    dfx = pd.read_hdf(p, key = 'expression')
    dfy = pd.read_hdf(p, key = 'labels')
    print(p, dfy.value_counts().to_dict())

    outer = KFold(n_splits = n_fold, shuffle = True, random_state = 123)
    outer_idx = list(outer.split(dfx, dfy))

    fold_all = []
    for i, idx in enumerate(outer_idx):

        fold_num = "fold_%s" % str(i).zfill(2) 

        train_idx, test_idx = idx

        trainX = dfx.iloc[train_idx].values
        trainY = dfy.iloc[train_idx].values

        testX = dfx.iloc[test_idx].values
        testY = dfy.iloc[test_idx].values

        ## training
        print("\nInput train and test X shape is %s, %s  \n" % (trainX.shape,  testX.shape))
        print("Getting the best parameters by gridsearch \n")
        best_params = get_best_params(trainX, trainY)
        clf = xgb.XGBClassifier(eval_metric='logloss', **best_params, n_jobs=-1)
        clf.fit(trainX, trainY)

        ## evaluation
        y_true = testY
        y_score = clf.predict_proba(testX)[:,1]
        dfr = pd.DataFrame([y_true, y_score]).T
        dfr.columns = ['y_true', 'y_score']
        dfr.index = dfy.iloc[test_idx].index
        acc, auc = score(dfr)
        
        fold_all.append(auc)
        res.append([auc, i, p, best_params])
        
    print('The auc score for %s is %s.' % (p, np.mean(fold_all)))


################################################## /raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_test_HNSC_grade.h5 ##################################################
/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_test_HNSC_grade.h5 {0: 385, 1: 119}


100%|#################################################################################| 504/504 [00:38<00:00, 12.94it/s]



Input train and test X shape is (403, 17970), (101, 17970)  

Getting the best parameters by gridsearch 

[11:13:25] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


acc: 0.782, roc-auc: 0.808, BER: 0.418

Input train and test X shape is (403, 17970), (101, 17970)  

Getting the best parameters by gridsearch 

[11:17:20] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


acc: 0.762, roc-auc: 0.581, BER: 0.414

Input train and test X shape 

/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The u

[11:21:37] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


acc: 0.802, roc-auc: 0.806, BER: 0.401

Input train and test X shape is (403, 17970), (101, 17970)  

Getting the best parameters by gridsearch 



...

[13:34:14] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:34:58] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:35:33] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you fin

/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The u

[13:49:59] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


acc: 0.640, roc-auc: 0.697, BER: 0.427

Input train and test X shape is (300, 17970), (74, 17970)  

Getting the best parameters by gridsearch 

[13:34:14] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:34:58] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language b

/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The u

[13:34:14] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:34:39] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:36:00] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you fin

In [6]:
pd.DataFrame(res)

,0,1,2,3
0,0.808400,0,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 4, 'n_estimators': 100}"
1,0.581169,1,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 4, 'n_estimators': 100}"
2,0.806022,2,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 6, 'n_estimators': 100}"
3,0.735263,3,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 9, 'n_estimators': 100}"
4,0.831230,4,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 6, 'n_estimators': 100}"
5,0.710938,0,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 4, 'n_estimators': 100}"
6,0.618729,1,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 6, 'n_estimators': 50}"
7,0.687243,2,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 4, 'n_estimators': 20}"
8,0.467980,3,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 6, 'n_estimators': 50}"
9,0.626667,4,/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_...,"{'max_depth': 4, 'n_estimators': 20}"


In [7]:
dfres = pd.DataFrame(res, columns = ['score', 'fold', 'task', 'best_params'])
dfres.to_csv('./xgboost_5FCV_results_grade.csv')
dfres.groupby('task').mean().sort_values('score',ascending=False)

,score,fold
task,,
/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_train_UCEC_grade.h5,0.882266,2.0
/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_test_HNSC_grade.h5,0.752417,2.0
/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_train_STAD_grade.h5,0.746808,2.0
/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_train_LGG_grade.h5,0.736230,2.0
/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_train_LIHC_grade.h5,0.650754,2.0
/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_train_PAAD_grade.h5,0.622311,2.0
/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_train_CESC_grade.h5,0.582557,2.0
/raid/shenwanxiang/AggMapNet_dataset/TCGA-G/O_z-score_train_KIRC_grade.h5,0.578386,2.0


/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The u

[13:52:12] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:52:36] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:50:37] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you fin

/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The u

[13:34:15] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:34:56] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:36:14] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you fin

...

/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The u

[13:52:48] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.




/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The u

[13:52:52] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:34:14] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you find above cases.


[13:34:38] WARNING: ../src/learner.cc:573: 
Parameters: { "silent", "verbose" } might not be used.

  This may not be accurate due to some parameters are only used in language bindings but
  passed down to XGBoost core.  Or some parameters are not used but slip through this
  verification. Please open an issue if you fin

/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The use of label encoder in XGBClassifier is deprecated and will be removed in a future release. To remove this warning, do the following: 1) Pass option use_label_encoder=False when constructing XGBClassifier object; and 2) Encode your labels (y) as integers starting with 0, i.e. 0, 1, 2, ..., [num_class - 1].
  warnings.warn(label_encoder_deprecation_msg, UserWarning)
/home/sxh/.local/lib/python3.8/site-packages/xgboost/sklearn.py:1146: UserWarning: The u